In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

C:\Users\User\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
import numpy as np

def bootstrap_resample(data, num_iterations):
    resampled_models = []
    n = len(data)
    
    for _ in range(num_iterations):
        # Generate a bootstrap sample by randomly sampling with replacement
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        
        # Append the bootstrap sample to the list of resampled models
        resampled_models.append(bootstrap_sample)
    
    return resampled_models

In [3]:
data = np.array(['train_data_19_03.xlsx'])  # Replace this with your dataset
num_resampled_models = 500

# Perform bootstrap resampling
resampled_models = bootstrap_resample(data, num_resampled_models)

# Print the first resampled model (as an example)
print("Example of a resampled model:")
print(resampled_models[0])


Example of a resampled model:
['train_data_19_03.xlsx']


In [4]:
train_df = pd.read_excel('train_data_19_03.xlsx', index_col=False)

In [5]:
test_df = pd.read_excel('test_data_19_03.xlsx', index_col=False)

In [6]:
X_train = train_df[train_df.columns[:-1]]
y_train = train_df[train_df.columns[-1]]

X_test = test_df[test_df.columns[:-1]]
y_test = test_df[test_df.columns[-1]]

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3969 entries, 0 to 3968
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Hm0 d     3969 non-null   float64
 1   Tp d      3969 non-null   float64
 2   Tm d      3969 non-null   float64
 3   Tm1,0 d   3969 non-null   float64
 4   hdeep     3969 non-null   float64
 5   mmm       3969 non-null   float64
 6   β         3969 non-null   float64
 7   Spread s  3969 non-null   float64
 8   h         3969 non-null   float64
 9   Hm0 toe   3969 non-null   float64
 10  Tp toe    3969 non-null   float64
 11  Tm toe    3969 non-null   float64
 12  Tm1,0t    3969 non-null   float64
 13  ht        3969 non-null   float64
 14  Bt        3969 non-null   float64
 15  cotαd     3969 non-null   float64
 16  cotαu     3969 non-null   float64
 17  cotαexcl  3969 non-null   float64
 18  cotαincl  3969 non-null   float64
 19  gf_d      3969 non-null   float64
 20  D50_d     3969 non-null   floa

In [8]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 3969 entries, 0 to 3968
Series name: log_q
Non-Null Count  Dtype  
--------------  -----  
3969 non-null   float64
dtypes: float64(1)
memory usage: 31.1 KB


In [10]:
num_resampled_models = 500

# Perform bootstrap resampling
resampled_models = bootstrap_resample(train_df, num_resampled_models)


ValueError: a must be 1-dimensional

In [9]:
regressor = xgb.XGBRegressor()

In [10]:
xg_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    learning_rate = 0.07,
    max_depth = 21, 
    min_child_weight=3,
    reg_lambda=1,
    subsample=0.75
)

In [24]:
xg_reg.fit(X_train, y_train)

C:\Users\User\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.07, max_delta_step=0,
             max_depth=21, min_child_weight=3, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.75,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
def bootstrap_resample(data, num_iterations):
    resampled_models = []
    n = len(data)
    
    for _ in range(num_iterations):
        # Generate random indices for resampling with replacement
        indices = np.random.choice(range(n), size=n, replace=True)
        
        # Create the bootstrap sample by selecting rows based on the indices
        bootstrap_sample = data[indices]
        
        # Append the bootstrap sample to the list of resampled models
        resampled_models.append(bootstrap_sample)
    
    return resampled_models

def build_model_and_calculate_rmse(training_data, testing_data):
    X_train = training_data[:, :-1]
    y_train = training_data[:, -1]
    X_test = testing_data[:, :-1]
    y_test = testing_data[:, -1]
    
    # Build a model (you can use any model you like)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

# Example usage:
train_data = pd.read_excel('train_data_19_03.xlsx').values
test_data = pd.read_excel('test_data_19_03.xlsx').values

num_resampled_models = 500

# Perform bootstrap resampling
resampled_models = bootstrap_resample(train_data, num_resampled_models)

# Calculate RMSE for each resampled model
rmse_values = []
for resampled_model in resampled_models:
    rmse = build_model_and_calculate_rmse(resampled_model, test_data)
    rmse_values.append(rmse)

# Print the average RMSE over all resampled models
average_rmse = np.mean(rmse_values)
print("Average RMSE:", average_rmse)

Average RMSE: 0.6530035027524678


In [18]:
best_rmse = np.min(rmse_values)
print("Best RMSE:", best_rmse)

Best RMSE: 0.6367148023735614


In [35]:
    X_train = training_data[:, :-1]
    y_train = training_data[:, -1]
    X_test = testing_data[:, :-1]
    y_test = testing_data[:, -1]

NameError: name 'training_data' is not defined

In [27]:
regressor = xgb.XGBRegressor()

In [28]:
xg_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    learning_rate = 0.07,
    max_depth = 21, 
    min_child_weight=3,
    reg_lambda=1,
    subsample=0.75
)

In [33]:
xg_reg.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [19]:
def calculate_percentage_error(actual, predicted):
    absolute_error = np.abs(actual - predicted)
    relative_error = absolute_error / actual
    percentage_error = relative_error * 100
    return percentage_error

In [26]:
mape = mean_absolute_percentage_error(resampled_model, test_data)

ValueError: Found input variables with inconsistent numbers of samples: [3969, 1701]